In [ ]:
!pip install --upgrade transformers sentence-transformers

In [ ]:
!pip install torch scikit-learn pandas streamlit


In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import re


In [ ]:
def clean_text(text):
    text = re.sub(r'\W', ' ', text)  # Remove special chars
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)  # Remove single characters
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)  # Remove single characters at start
    text = re.sub(r'\s+', ' ', text)  # Remove multiple spaces
    return text.strip().lower()


In [ ]:
true_df = pd.read_csv("true.csv").head(100)
fake_df = pd.read_csv("fake.csv").head(100)

true_df['label'] = 1
fake_df['label'] = 0

df = pd.concat([true_df, fake_df], ignore_index=True)
df['text'] = df['title'] + " " + df['text']
df['text'] = df['text'].apply(clean_text)

df = df[['text', 'label']]
df.head()


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [ ]:
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df['text'], df['label'], test_size=0.2, stratify=df['label'], random_state=42)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, stratify=temp_labels, random_state=42)


In [ ]:
from torch.utils.data import Dataset

class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_len)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
train_dataset = NewsDataset(train_texts.tolist(), train_labels.tolist(), tokenizer)
val_dataset = NewsDataset(val_texts.tolist(), val_labels.tolist(), tokenizer)
test_dataset = NewsDataset(test_texts.tolist(), test_labels.tolist(), tokenizer)


In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",  # Changed from "steps" to "epoch"
    logging_dir='./logs',
    load_best_model_at_end=True,
    save_total_limit=1
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


In [ ]:
trainer.train()


In [ ]:
trainer.evaluate(test_dataset)


In [ ]:
def predict_news(text, model, tokenizer):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)
        prediction = torch.argmax(probs).item()
        confidence = torch.max(probs).item()
    return prediction, confidence


In [ ]:
# Try with a sample news article
sample_text = "The government has announced a new policy for electric vehicles to boost green energy."
label, confidence = predict_news(sample_text, model, tokenizer)

print("Prediction:", "Real News 🟢" if label == 1 else "Fake News 🔴")
print("Confidence:", round(confidence * 100, 2), "%")


In [ ]:
model.save_pretrained("./fake_news_model")
tokenizer.save_pretrained("./fake_news_model")


In [ ]:
import shutil
shutil.make_archive("fake_news_model", 'zip', "fake_news_model")


In [ ]:
from google.colab import files
files.download("fake_news_model.zip")
